In [23]:
from notebook_utils import setup
import pandas as pd
import networkx as nx
from collections import defaultdict
import numpy as np
import pickle
import json

setup()
DATAFRAMES_DIR = "../data/dataframes/16-dec/"

In [24]:
with open("../data/handleToCandidateInfo.json", "r") as f:
    candidates = json.load(f)

In [25]:
candidate_mapping = {}
for candidate in candidates:
    del candidate['']
    candidate["id"] = str(candidate["id"])
    candidate_mapping[candidate["id"]] = candidate

list(candidate_mapping.items())[0]

('941080085121175600',
 {'handle': 'sendougjones',
  'name': 'Senator Doug Jones',
  'verified': 'True',
  'id': '941080085121175600',
  'state': 'Alabama',
  'party': 'Democratic',
  'position': 'U.S. Senate',
  'candidate_name': 'Doug Jones (Alabama)'})

In [26]:
candidate_mapping["25073877"] = {
    "handle": "realDonaldTrump",
    "name": "Donald Trump",
    "party": "Republican",
    "position": "President",
    "state": "N/A",
    "id": "25073877",
    "verified": True,
    "candidate_name": "N/A"
}

In [27]:
df_retweets = pd.read_pickle("./df_retweets_with_cluster.pickle")

In [28]:
candidate_shares = defaultdict(lambda: [])

for i, user, retweeted in df_retweets[["user", "retweeted"]].itertuples():
    if (user in candidate_mapping):
        candidate_shares[user].append(retweeted)

    if (i % 100000 == 0):
        print("Processed", i)

Processed 0
Processed 100000
Processed 200000
Processed 300000
Processed 400000
Processed 500000
Processed 600000
Processed 700000
Processed 800000
Processed 900000
Processed 1000000
Processed 1100000
Processed 1200000
Processed 1300000
Processed 1400000
Processed 1500000
Processed 1600000
Processed 1700000
Processed 1800000
Processed 1900000
Processed 2000000
Processed 2100000
Processed 2200000
Processed 2300000
Processed 2400000
Processed 2500000
Processed 2600000
Processed 2700000
Processed 2800000
Processed 2900000
Processed 3000000
Processed 3100000
Processed 3200000
Processed 3300000
Processed 3400000
Processed 3500000
Processed 3600000
Processed 3700000
Processed 3800000
Processed 3900000
Processed 4000000
Processed 4100000
Processed 4200000
Processed 4300000
Processed 4400000
Processed 4500000
Processed 4600000
Processed 4700000
Processed 4800000
Processed 4900000
Processed 5000000
Processed 5100000
Processed 5200000
Processed 5300000
Processed 5400000
Processed 5500000
Process

In [38]:
active_candidates = 0
for user_id, shares in candidate_shares.items():
    #print(shares)
    if (len(shares) > 0):
        active_candidates += 1

print("Total candidates", len(candidate_mapping))
print("Retweeting candidates", len(candidate_shares))
print("Trump shares", len(candidate_shares["25073877"]))

Total candidates 2367
Retweeting candidates 133
Trump shares 30


In [37]:
with open("./candidate_shares.pickle", "wb") as f:
    pickle.dump({
        "candidate_shares": dict(candidate_shares),
        "candidate_map": candidate_mapping
    }, f)